In [15]:
import pandas as pd
import numpy as np
import os
import hashlib
import requests
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse
from tqdm.notebook import tqdm
import random
from multiprocessing import Pool
import spacy

In [8]:
industries = pd.read_csv("industry_categories.csv")
industries.head()

,id,industry
0,1,Accounting/Finance
1,2,Administrative
2,3,Arts/Entertainment/Publishing
3,4,Banking/Loans
4,5,Construction/Facilities


In [9]:
salary_industries = pd.read_csv("Salary-Industries.csv")
salary_industries.head()

,Industry Clean,Mapped,count,Industry Ref
0,Hospital & Health Care,True,1421,Healthcare
1,Primary & Secondary Education,True,1142,Education
2,Non-Profit Organization Management,True,1135,Nonprofit
3,Higher Education,True,994,Higher Education
4,Technology,NaN,799,Technology


In [11]:
industry_targets_urls = pd.read_csv("utils/industry_targets_urls.csv")
industry_targets_urls.head()

,industry,term,url
0,Accounting/Finance,Accounting,https://www.coursera.org/learn/uva-darden-fina...
1,Accounting/Finance,Accounting,https://www.accounting-degree.org/what-is-acco...
2,Accounting/Finance,Accounting,https://www.accountingcoach.com/accounting-bas...
3,Accounting/Finance,Accounting,https://en.wikipedia.org/wiki/Accounting
4,Accounting/Finance,Accounting,https://www.accounting.com/


In [12]:
industry_inputs_urls = pd.read_csv("utils/industry_inputs_urls.csv")
industry_inputs_urls.head()

,industry,term,url
0,(blank),(blank),https://www.merriam-webster.com/dictionary/blank
1,(blank),(blank),http://blank.org/
2,(blank),(blank),https://www.dictionary.com/browse/blank
3,(blank),(blank),https://en.wiktionary.org/wiki/blank
4,(blank),(blank),https://en.wikipedia.org/wiki/Blank_(2019_film)


In [13]:
industry_targets_text = pd.read_csv("utils/industry_targets_text.csv")
industry_targets_text.head()

,industry,term,text
0,Accounting/Finance,Accounting,This course will teach you the tools you'll ne...
1,Accounting/Finance,Accounting Finance,How Much House Can I Afford? What is the Best ...
2,Accounting/Finance,Finance,if you are not redirected within a few seconds...
3,Administrative,Administrative,Gain access to thousands of additional definit...
4,Agriculture,Agriculture,National Geographic Society is a 501 (c)(3) or...


In [37]:
industry_inputs_text = pd.read_csv("utils/industry_inputs_text.csv")
industry_inputs_text

,industry,term,text
0,(blank),(blank),Gain access to thousands of additional definit...
1,25,25,. Unsourced material may be challenged and rem...
2,501(c)3 nonprofit - homeless services,501(c)3 nonprofit - homeless services,Why Are So Many People Homeless? Why Are So Ma...
3,A/E,A/E,New Episodes Tonight & Tomorrow 9P Watch Every...
4,A/E/C Consulting,A/E/C Consulting,“We truly appreciate your level of commitment ...
...,...,...,...
5027,transportation/education,transportation/education,Program usually offered at the college level 4...
5028,trucking/transportation,trucking/transportation,nan Click here to continue to the news NACV to...
5029,user experience/product design,user experience/product design,switch to a modern web browser to improve perf...
5030,video game marketing,video game marketing,30 of the Most Successful Video Game Marketing...


In [50]:
# nlp = spacy.load("en_core_web_sm", disable=[
#     "tagger", "parser", "ner", "textcat", "entity_linker", "entity_ruler", "sentencizer", "merge_noun_chunks",
#     "merge_entities", "merge_subtokens"])
# industry_targets_docs = list(nlp.pipe(tqdm(industry_targets_text['text'][:100])))

In [51]:
# for token in industry_targets_docs[1]:
#      print(token.text)

In [ ]:
# print(len(industry_docs))
# Y = np.concatenate([np.reshape(d.vector, (1, -1)) for d in industry_docs])
# print(Y.shape)

# X = np.concatenate([np.reshape(d.vector, (1, -1)) for d in industry_inputs_docs])
# print(X.shape)

In [66]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords

from sklearn.base import BaseEstimator, TransformerMixin

class NltkLemmatizer(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.stopwords = set(stopwords.words('english'))
        self.lemmer=WordNetLemmatizer()

    def fit(self, X, y=None):
        return self  # generally does nothing

    def transform(self, X, y=None):
        return [' '.join([self.lemmer.lemmatize(word) for word in doc.split(' ') if word not in self.stopwords]) for doc in X]

s = industry_targets_text.loc[1, 'text']    
l = NltkLemmatizer().fit_transform([s])
# print(s[:2000])
# print(normalizer()(s)[:2000])
# print(stopwords.words('english'))
# print(s[:1000])
# print(word_tokenize(s)[:200])
# stemmer = SnowballStemmer('english')
# print(' '.join([stemmer.stem(word) for word in word_tokenize(s)][:100]))

# lemmer=WordNetLemmatizer()
# print(' '.join([lemmer.lemmatize(word) for word in word_tokenize(s)][:100]))
# cntv = CountVectorizer(lowercase=True, stop_words='english')
# x = cntv.fit_transform([s])
# print(cntv.get_stop_words())
# print(cntv.get_feature_names())
# print(x.toarray()) 

In [68]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
text_clf = Pipeline([
    ('lemmer', NltkLemmatizer()),
    ('vect', CountVectorizer(strip_accents='ascii', lowercase=True, stop_words='english')),
    ('tfidf', TfidfTransformer())
])
Y = text_clf.fit_transform(tqdm(industry_targets_text.loc[:,'text'], desc='Y'))
X = text_clf.transform(tqdm(industry_inputs_text.loc[:,'text'], desc='X'))
print(Y.shape)
print(X.shape)


(87, 38829)
(5032, 38829)


In [69]:
from sklearn.metrics import pairwise_distances_argmin_min

Y_indx, _ = pairwise_distances_argmin_min(X, Y, metric='euclidean')
euclidean_matches = industry_targets_text.loc[Y_indx, ['industry', 'term']].reset_index()
Y_indx, _ = pairwise_distances_argmin_min(X, Y, metric='cosine')
cosine_matches = industry_targets_text.loc[Y_indx, ['industry', 'term']].reset_index()


In [74]:
results = pd.DataFrame({
    'input_term': industry_inputs_text.loc[:, 'term'],
    'term': euclidean_matches['term'],
    'industry': euclidean_matches['industry'],
    #'cosine_industry': cosine_matches['industry'],
    #'cosine_term': cosine_matches['term'],
    #'l2_industry': l2_matches['industry'],
    #'l2_term': l2_matches['term']
})
#results[results['euclidean_term'] != results['cosine_term']]


In [79]:
from IPython.core.display import display, HTML
display(HTML(results.to_html()))

,input_term,term,industry
0,(blank),Media,Media
1,25,PR,Marketing/Advertising/PR
2,501(c)3 nonprofit - homeless services,Non-Profit,Non-Profit/Volunteering
3,A/E,Community,Social/Community
4,A/E/C Consulting,Consulting,Upper Management/Consulting
5,AEC,Engineering,Engineering/Architecture
6,AECO,Gas,Oil/Gas/Energy
7,AI,Research,Science/Research
8,AI/Biotech,Pharmaceutical Bio-tech,Pharmaceutical/Bio-tech
9,ATM Industry,Banking,Banking/Loans


In [81]:
results.to_csv("industries_classified.csv", index=False)